In [1]:
import getpass
import os


In [2]:
import requests

def lobstr_search_function(query, api_key, max_results=2):
    crawler_hash = "google-serp-hash"  
    base_url = 'https://api.lobstr.io/v1/'
    headers = {'Authorization': f'Token {api_key}'}
    
    create_squid_payload = {
        "crawler": crawler_hash,
        "inputs": {
            "query": query,
        }
    }
    response = requests.post(
        base_url + 'squids', 
        json=create_squid_payload, 
        headers=headers
    )
    if not response.ok:
        return None
    squid_id = response.json().get('id')
    if not squid_id:
        return None
    for _ in range(10):
        result_resp = requests.get(base_url + f'squids/{squid_id}', headers=headers)
        result_data = result_resp.json()
        if result_data.get("status") == "finished":
            results = result_data.get('results', [])
            return results[:max_results] if results else None
        import time; time.sleep(1)
    return None


In [4]:
from langchain_tavily import TavilySearch
from langchain_community.utilities import SerpAPIWrapper
from langchain.agents import Tool
import os
serp_search = SerpAPIWrapper(serpapi_api_key=getpass.getpass("Enter serpAPI key:"))
tavily_search = TavilySearch(max_results = 2, tavily_api_key = getpass.getpass("Enter tavily API key:"))
lobstr_api_key = getpass.getpass("Enter lobstr key:")
def lobstr_search(query):
    return lobstr_search_function(query, api_key= lobstr_api_key , max_results=2)
serp_tool = Tool(
    name="SerpAPI Search",
    description="Search the web using SerpAPI",
    func=serp_search.run
)
tavily_tool = Tool(
    name="Tavily Search",
    description="Search the web using Tavily",
    func=tavily_search.run
)
lobstr_tool = Tool(
    name="lobstr_web_search",
    description="Search the web via Lobstr SERP API",
    func=lambda query: lobstr_search(query, api_key= lobstr_api_key, max_results=2)
)


In [6]:
def safe_search(search_func, query):
    try:
        result = search_func(query)
        if result is None or result == "" or (
            isinstance(result, dict) and ("error" in result or "Error" in result)) or (
            isinstance(result, str) and "Error" in result):
            return None
        return result
    except Exception:
        return None

In [7]:
def combined_search(query):
    result = safe_search(tavily_search.run, query)
    if result:
        return f"Tavily result:\n{result}"
    result = safe_search(serp_search.run, query)
    if result:
        return f"SerpAPI result:\n{result}"
    result = safe_search(lobstr_search, query)
    if result:
        return f"Lobstr result:\n{result}"
    return "No information found from Tavily, SerpAPI and lobstr"
    
fallback_tool = Tool(
    name="Combined_web_search",
    description="Try Tavily, fallback to SerpAPI and lobstr if needed.",
    func=combined_search
)

In [8]:
tools = [fallback_tool]

In [10]:
os.environ["MISTRAL_API_KEY"] = getpass.getpass("Enter API key for Mistral: ")
api = os.environ["MISTRAL_API_KEY"]
from langchain_mistralai import ChatMistralAI
import os
llm = ChatMistralAI(model="mistral-small-latest", api_key=api)

In [11]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools)

In [12]:
input_message = {
    "role": "user",
    "content": "Whats the weather in bangalore",
}

for step in agent_executor.stream(
    {"messages": [input_message]}, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Whats the weather in bangalore
================================== Ai Message ==================================
Tool Calls:
  Combined_web_search (0SO8NbGFg)
 Call ID: 0SO8NbGFg
  Args:
    __arg1: weather in bangalore
================================= Tool Message =================================
Name: Combined_web_search

Tavily result:
{'query': 'weather in bangalore', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Weather in Bangalore', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Bangalore', 'region': 'Karnataka', 'country': 'India', 'lat': 12.9833, 'lon': 77.5833, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1755349870, 'localtime': '2025-08-16 18:41'}, 'current': {'last_updated_epoch': 1755349200, 'last_updated': '2025-08-16 18:30', 'temp_c': 23.1, 'temp_f': 73.6, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weathe